# Projet 3 - Among Us

---

On utilise le fichier compressé `AmongUs.7z` où l'ensemble des fichiers décrivent les statistiques des 
parties jouées sur le jeu *Among Us*. Il y a un fichier par joueur nommé *UserX.csv*.

Les fichiers csv ont tous la même structure avec :

- `Game.Completed.Date` : Date de la partie
- `Team` : l'équipe attribuée
- `Outcome` : résultat de la partie
- `Task.Completed` : le nombre de tâches effectuées
- `All.Tasks.Completed` : si toutes les tâches ont été effectuées
- `Murdered` : si le joueur a été tué
- `Imposter.Kills` : le nombre de joueurs tués par l'imposteur
- `Game.Length` : durée de la partie
- `Ejected` : si le joueur a été éliminé par les autres au cours de la partie
- `Sabotages.Fixed` : nombre de sabotages réparés
- `Time.to.complete.all.tasks` : temps pour compléter les toutes les tâches
- `Rank.Change` : Non renseigné
- `Region.Game.Code` : la région du serveur de jeu

---
---

## Exercice 1 : Créer le jeu de données

---

### a. Télécharger le fichier compressé `AmongUs.7z` et le déziper. 

La fonction `list.files()` devrait vous aider à extraire l'ensemble des noms de fichiers présents dans un répertoire (voir l'exemple ci-dessous).
```{r}
list.files(path = "dataset/", pattern="*.csv", full.names=FALSE)
```


In [ ]:
files <- list.files(path = "./AmongUs/", pattern = "*.csv", full.names = FALSE)
files

---

### b. Compiler l'ensemble des fichiers *UserX.csv* présents dans le fichier zip dans un seul et même data frame. Utiliser une boucle qui parcourt chaque fichier csv utilisateur. 
La fonction `rbind()` devrait vous aider à compiler les différents csv dans un seul data frame (voir l'exemple ci-dessous). 

```{r}
df <- data.frame()
dim(df)
df <- rbind(df, mtcars)
dim(df)
df <- rbind(df, mtcars)
dim(df)
```
Attention, pour le bon déroulement du Brief, intégrer l'argument `stringsAsFactors = FALSE` dans votre fonction `read.csv()`. Cet argument permet de ne pas typer par défaut les variables `character` en `factor` lors de l'importation.


In [ ]:
df <- data.frame()
for (file in files){
    df <- rbind(df, read.csv(paste0("./AmongUs/", file), stringsAsFactors = FALSE))
}
df

---

###  c. Modifier votre boucle afin d'ajouter une colonne dans le data frame final qui renseigne le numéro d'utilisateur disponible dans le nom de chaque fichier csv.

indice : Il est plus simple d'ajouter une colonne avec le nom du fichier juste avant la fonction `rbind()`.

In [ ]:
df <- data.frame()
for (file in files){
    name <- substr(file, 1, nchar(file)-4)
    df_temp <- read.csv(paste0("./AmongUs/", file), stringsAsFactors = FALSE)
    df_temp$User <- name
    df <- rbind(df, df_temp) 
}
df


---

### d. Stocker le data frame final dans un objet appelé *AmongUs*.

In [ ]:
AmongUs <- data.frame(df)

---

### e. Supprimer la colonne `Rank.Change` qui est inutile.

In [ ]:
AmongUs = subset(AmongUs, select = -Rank.Change )
AmongUs

---

### f. On observe des valeurs manquantes dans le dataset à travers les champs `-` et `N/A`, on souhaite remplacer ces valeurs textuelles par la véritable valeur `NA` permettant d'indiquer à R que la valeur est manquante.

Construire une fonction appelée `manage_na()` qui prend en entrée :
- un vecteur `x` correspondra à une colonne d'un dataset.
- un vecteur `string_to_na` correspondant à la liste des chaînes de caractères qu'on souhaite transformer en `NA`.

La fonction retournera le vecteur modifié.

indice : voici ci-dessous la tête que doit avoir votre fonction :
```{r, eval=FALSE}
manage_na <- function(x, string_to_na) {
 
 ...
 ...
 ...
 
 return(x_modif)
}
```

In [ ]:
manage_na <- function(x, string_to_na){
    
    for (mot in string_to_na){
        
      x[x == mot] <- NA  
    }

return(x)
}

---

### g. Construire une boucle qui parcourt toutes les colonnes du data frame *AmongUs* et qui remplace les champs avec des `-` et `N/A` par des valeurs manquantes `NA`. Utilisez la fonction `manage_na()` créée précédemment.


In [ ]:
for (i in 1:ncol(AmongUs)){
    
    AmongUs[i] <- manage_na(AmongUs[i], c("-", "N/A"))
}

AmongUs

---

### h. Suppression des symboles dans le nom de la première colonne

In [ ]:
colnames(AmongUs)[1] <- gsub('^...','',colnames(AmongUs)[1])
AmongUs

---
---

## Exercice 2 : Manipuler le texte et les dates

---

### a. À partir de la colonne `Game.Completed.Date`, construire une colonne appelée `Date` avec la date de la partie au format *yyyy-mm-dd* uniquement. Veiller à ce que le type de cette colonne soit *Date*.

In [ ]:
library(lubridate)

In [ ]:
AmongUs$Date <- lapply(AmongUs$Game.Completed.Date, function (x) substr(x, 1, 10))
                       
AmongUs$Date <- mdy(AmongUs$Date)
                                                
AmongUs

---

### b. À partir de la colonne `Game.Completed.Date`, construire une colonne appelée `Heure` avec **l'heure uniquement** de la partie. Attention au format *am* et *pm*. 

In [ ]:
AmongUs$Heure <- lapply(AmongUs$Game.Completed.Date, function (x) substr(x, 15, 25))

convert_time <- function(time) {
    is_pm <- (substring(time, regexpr("\\s", time)[[1]] + 1, regexpr("\\s", time)[[1]] + 2) == "pm")
    if (is_pm) {
        result <- (as.POSIXct(substring(time, 1, regexpr("\\s", time)[[1]] - 1), format = "%H:%M:%S") + hours(12))
        return(strftime(result, format="%H:%M:%S"))
    } 
    else {
        result <- (as.POSIXct(substring(time, 1, regexpr("\\s", time)[[1]] - 1), format = "%H:%M:%S"))
        return(strftime(result, format="%H:%M:%S"))
    }
}   

                 
AmongUs$Heure <- hms(lapply(AmongUs$Heure, convert_time))
                
AmongUs

---

### c. À partir de la colonne `Game Length`, construire une colonne appelée `Game.Length.sec` correspondant à la durée de la partie en secondes. 

In [ ]:
AmongUs$Game.Length.sec <-  as.integer(substr(AmongUs$Game.Length, 1, 2)) * 60 + as.integer(substr(AmongUs$Game.Length, 4, 5))

AmongUs

---

### d. À partir de la colonne `Time.to.complete.all.tasks`, construire une colonne appelée `Complete.all.tasks.sec` correspondant à la durée en secondes pour compléter toutes les tâches.

In [ ]:
AmongUs$Complete.all.tasks.sec <-  as.integer(substr(AmongUs$Time.to.complete.all.tasks, 1, 2)) * 60 + as.integer(substr(AmongUs$Time.to.complete.all.tasks, 4, 5))

AmongUs

---

### e. À partir de la colonne `Region.Game.Code` construire une colonne appelée `Region` correspondant au nom du continent uniquement.

In [ ]:
AmongUs$Region <- substr(AmongUs$Region.Game.Code, 1, regexpr(" ", AmongUs$Region.Game.Code))

AmongUs$Region <- gsub("NA", "North America", AmongUs$Region)

AmongUs

---
---

## Exercice 3 : Type des variables

---

### a. Combien de lignes, colonnes sont présentes dans ce dataset (Utiliser la fonction adaptée) ?

In [ ]:
sprintf("Nombre de lignes: %s", dim(AmongUs)[1])
sprintf("Nombre de colonnes: %s", dim(AmongUs)[2])

---

### b. Afficher un résumé des données avec la fonction adaptée. 

In [ ]:
summary(AmongUs)

---

### c. Veiller à ce que les types de chaque colonne du dataset correspondent aux types ci-dessous. Sinon, convertir les variables dans leur type approprié.

Variable | Type souhaité
------------ | ------------
Game.Completed.Date | character
Team | factor
Outcome| factor
Task.Completed| numeric
All.Tasks.Completed| factor
Murdered| factor
Imposter.Kills| numeric
Game.Length| character
Ejected| factor
Sabotages.Fixed| numeric 
Time.to.complete.all.tasks| character
Region.Game.Code| character
**Date** | Date
**Heure** | numeric
**Game.Length.sec** | numeric
**Complete.all.tasks.sec** | numeric
**Region** | factor

<br>

Attention, lorsqu'on convertit un vecteur de type `factor` en type `numeric`, il est recommandé de passer d'abord par le type `character` (voir exemple ci-dessous). Ce ne sera peu être pas utile pour ce TP, mais c'est bien de le savoir !

Ce qu'il ne faut pas faire 
```{r}
x <- factor(c(7,7,8,7,9,6,6))
levels(x)
x <- as.numeric(x)
x
```
Ce qu'il faut faire 
```{r}
x <- factor(c(7,7,8,7,9,6,6))
levels(x)
x <- as.numeric(as.character(x))
x
```

In [ ]:
sapply(AmongUs, class)

In [ ]:
names <- c('Team' ,'Outcome', 'All.Tasks.Completed', 'Murdered', 'Ejected', 'Region')
AmongUs[names] <- lapply(AmongUs[names], factor)

names <- c('Task.Completed' ,'Imposter.Kills', 'Sabotages.Fixed', 'Heure')
AmongUs[names] <- lapply(AmongUs[names], as.numeric)

---

### e. Vérifier si cela a fonctionné en affichant le type de chaque variable du data frame.

In [ ]:
sapply(AmongUs, class)

---
---

## Exercice 4 : Analyses statistiques

La plupart des questions de cet exercice demandent un peu de réflexion. <br>
Ne partez pas à l'abordage, les solutions peuvent se coder en plusieurs étapes (*tris*,*filtres*, 
*agregations*, *etc.*).

---

### a. Quelle est la durée moyenne d'une partie ?

In [ ]:
min <- floor(mean(AmongUs$Game.Length.sec)/60)
sec <- floor(mean(AmongUs$Game.Length.sec) %% 60)

sprintf("Durée moyenne d'une partie : %smin %ssec", min, sec)

---

### b. Combien y-a-t-il de régions serveurs différentes ?.

In [ ]:
sprintf("Nombre de régions différentes : %s", length(unique(AmongUs$Region)))

---

### c. Combien de tâche maximum un Crewmate peut-il réaliser ?

In [ ]:
sprintf("Nombre de tâche maximum réalisable : %s", max(AmongUs$Task.Completed, na.rm = TRUE))

---

### d. Quel est le taux de parties remportées par les imposteurs ?

In [ ]:
Imp_win <- round((sum(AmongUs$Team == "Imposter" & AmongUs$Outcome == "Win") / sum(AmongUs$Team == "Imposter")) * 100, 2)

sprintf("Taux de victoire des imposteurs : %s%%", Imp_win)

---

### e. Construire **un graphique adapté** permettant de visualiser la répartition du nombre de parties jouées selon la`Region`.

In [ ]:
barplot(
    table(AmongUs$Region), 
    main = "Répartition du nombre de parties jouées selon la Region", 
    xlab = "Région", 
    ylab = "Nombre de partie", 
    col = "darkred",
    ylim = c(0,1600)
)

---

### f. Construire **un graphique adapté** permettant de visualiser la répartition des joueurs qui termine ou pas leurs tâches selon s'ils se font tuer ou pas.

In [ ]:
d_c <- sum(AmongUs$Team == "Crewmate" & AmongUs$All.Tasks.Completed == "Yes" & AmongUs$Murdered == "Yes")

d_i <- sum(AmongUs$Team == "Crewmate" & AmongUs$All.Tasks.Completed == "No" & AmongUs$Murdered == "Yes")

a_c <- sum(AmongUs$Team == "Crewmate" & AmongUs$All.Tasks.Completed == "Yes" & AmongUs$Murdered == "No")

a_i <- sum(AmongUs$Team == "Crewmate" & AmongUs$All.Tasks.Completed == "No" & AmongUs$Murdered == "No")


df = data.frame(
  x = c('Mort Fini', 'Mort Non-Fini', 'Vivant Fini', 'Vivant Non-fini'),
  y = c(d_c, d_i, a_c, a_i)
)

barplot(
    df$y, 
    names.arg=df$x, 
    ylab="Nombre", 
    main = "Répartition des joueurs qui termine ou pas leurs tâches \n selon s'ils se font tuer ou pas", 
    col = c("red", "darkred", "blue", "darkblue"),
    ylim = c(0,600)
)

---

### g. Construire **un graphique adapté** permettant de visualiser la distribution du nombre de tâches complétées par les joueurs.

In [ ]:
barplot(
    table(AmongUs$Task.Completed), 
    main = "Distribution du nombre de tâches complétées par les joueurs", 
    xlab = "Nombre de tâches", 
    ylab = "Comptage", 
    col = "darkred",
    ylim = c(0,500)
)

---

### h. Construire **un graphique adapté** permettant de visualiser pour chaque partie jouée, la durée de la partie et le temps pour compléter toutes les tâches.

In [ ]:
plot(
    AmongUs$Complete.all.tasks.sec, 
    AmongUs$Game.Length.sec, 
    main = "Durée de la partie en fonction du temps \n pour compléter toutes les tâches",
    xlab = "Temps pour finir les tâches", 
    ylab = "Temps de la partie", 
    col = "darkred"
)

---

### i. Construire **un graphique adapté** permettant de visualiser la distribution du temps des parties selon la `Region`.

In [ ]:
plot(
    AmongUs$Region, 
    AmongUs$Game.Length.sec, 
    main = "Distribution du temps des parties selon la Region",
    xlab = "Temps pour finir les tâches", 
    ylab = "Temps de la partie", 
    col = "darkred"
)

---

### j. Construire **un graphique adapté** permettant de visualiser l'évolution du nombre de parties jouées selon l'heure de la journée.

In [ ]:
barplot(
    table(floor(AmongUs$Heure / (60*60))),
    main = "Nombre de parties en fonction de l'heure de la journée", 
    xlab = "Heure", 
    ylab = "Nombre de parties", 
    col = "darkred",
    ylim = c(0,200)
)
    

---

### k. Construire **un graphique adapté** permettant de visualiser les variations du taux de succès des imposteurs selon les régions serveurs.

In [ ]:
Imp_win_eu <- round((sum(AmongUs$Team == "Imposter" & AmongUs$Outcome == "Win" & AmongUs$Region == "Europe ") / sum(AmongUs$Team == "Imposter" & AmongUs$Region == "Europe ")) * 100, 2)
Imp_win_na <- round((sum(AmongUs$Team == "Imposter" & AmongUs$Outcome == "Win" & AmongUs$Region != "Europe ") / sum(AmongUs$Team == "Imposter" & AmongUs$Region != "Europe ")) * 100, 2)

df <- data.frame(
    x = c("Europe", "North America"),
    y = c(Imp_win_eu, Imp_win_na)
)

barplot(
    df$y, 
    names.arg=df$x, 
    ylab="Nombre", 
    main = "Variations du taux de succès des imposteurs selon les régions serveurs", 
    col = c("darkred", "darkblue"),
    ylim = c(0,60)
)

---

### l. Construire **un graphique adapté** permettant de visualiser les taux de succès des 5 meilleurs utilisateurs.

In [ ]:
df <- data.frame(User = unique(AmongUs$User))
df$winrate <- as.numeric(lapply(df$User, function(x) round((sum(AmongUs$Outcome == "Win" & AmongUs$User == x) / sum(AmongUs$User == x)) * 100, 2)))

df <- df[order(-df$winrate),][1:5,]
barplot(                        
    df$winrate, 
    names.arg=df$User, 
    ylab="Winrate", 
    main = "Taux de succès des 5 meilleurs utilisateurs", 
    col = c('darkred'),
    ylim = c(0,100)
)
      
